In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import math
import librosa
import librosa.display

path="/content/drive/MyDrive/PRNN_A1/"
path_test =path+"test/"
path_train =path+"train/"
train_music= path+"SA1.WAV"
train_phoneme = path+"SA1.PHN"

In [ ]:
def get_phoneme_dict():
    phoneme_list=["bcl","dcl","gcl","pcl","tck","kcl","dcl","tcl","b","d","g","p","t","k","dx","q","jh","ch","s","sh","z","zh","f","th","v","dh","m","n","ng","em","en","eng","nx","l","r","w","y","hh","hv","el","iy","ih","eh","ey","ae","aa","aw","ay","ah","ao","oy","ow","uh","uw","ux","er","ax","ix","axr","ax-h","pau","epi","h#"]
    phonemlist_length = 63
    #create key value dictionary
    dict_timit = {}
    for ph in phoneme_list: 
        if('a' in ph or 'e' in ph or 'i' in ph or 'o' in ph or 'u' in ph):
          dict_timit[ph] = 0                                                ##phoneme is vowel
        else:
          dict_timit[ph] = 1                                                ##phoneme is consonent
    return dict_timit

phoneme=get_phoneme_dict()
print(phoneme)

{'bcl': 1, 'dcl': 1, 'gcl': 1, 'pcl': 1, 'tck': 1, 'kcl': 1, 'tcl': 1, 'b': 1, 'd': 1, 'g': 1, 'p': 1, 't': 1, 'k': 1, 'dx': 1, 'q': 1, 'jh': 1, 'ch': 1, 's': 1, 'sh': 1, 'z': 1, 'zh': 1, 'f': 1, 'th': 1, 'v': 1, 'dh': 1, 'm': 1, 'n': 1, 'ng': 1, 'em': 0, 'en': 0, 'eng': 0, 'nx': 1, 'l': 1, 'r': 1, 'w': 1, 'y': 1, 'hh': 1, 'hv': 1, 'el': 0, 'iy': 0, 'ih': 0, 'eh': 0, 'ey': 0, 'ae': 0, 'aa': 0, 'aw': 0, 'ay': 0, 'ah': 0, 'ao': 0, 'oy': 0, 'ow': 0, 'uh': 0, 'uw': 0, 'ux': 0, 'er': 0, 'ax': 0, 'ix': 0, 'axr': 0, 'ax-h': 0, 'pau': 0, 'epi': 0, 'h#': 1}


In [ ]:
def get_x_and_y(file_path,phoneme):
  x=[]
  y=[]
  count=0

  for folder in os.listdir(file_path):
    path = file_path + folder + "/"
    temp_name=""
    for files in os.listdir(path):
      name = files.split(".")[0]
      if name != temp_name:
        temp_name=name

        wav_file = path + name + ".WAV"
        phn_file = path + name + ".PHN"
        #print(wav_file,phn_file)

        data, sampling_freq = librosa.load(wav_file,sr=None, mono=True,offset=0.0,duration=None)
        data=data.tolist()

        file_obj = open(phn_file, 'r')
        phonem_data = file_obj.readlines()
        n=np.shape(phonem_data)[0]

        for i in range(n):
          lower,upper,ph=phonem_data[i].split(" ")

          lower=int(lower)
          upper=int(upper)
          ph=ph.replace("\n", "")

          temp=data[lower:upper]
          temp=np.array(temp)

          print(count)
          count+=1

          mfccs = librosa.feature.mfcc(temp, sr=sampling_freq)
          # mfccs=mfccs.flatten()
          temp=mfccs.tolist()

          x.append(temp)
          y.append(phoneme[ph])
          if(count==1000):
            return x,y

  return x,y

In [ ]:
def get_max_feature_len(x):
  max_len1=0
  max_len2=0
  n=x.__len__()
  for i in range(n):
    max_len1=max(max_len1,np.shape(x[i])[0])
    max_len2=max(max_len2,np.shape(x[i])[1])
  #print(max_len)
  return max_len1,max_len2

In [ ]:
def add_padding(x,max_len):
  x_train=[]
  n=x.__len__()
  # x_train=np.array(x_train)
  # for i in range(n):
  #   m=np.shape(x[i])[1]
  #   temp=np.zeros((20,max_len))
  #   if(m>max_len):
  #     temp=x[i][:][:max_len]
  #   else:
  #     temp[0:19][:m]=x[i]
    
  #   x_train.append(temp)
  for i in range(n):
    xs=np.zeros((20,max_len))
    # m=len(xs)
    t=np.shape(x[i])[1]
    for j in range(20):
      temp=np.array(x[i][j])
      if(t<max_len):
        xs[j,:t]=temp
      else:
        xs[j,:]=temp[:max_len]
    x_train.append(xs)
  return x_train
 

In [ ]:
def preprocessing_audio(path):
  
  path_test =path+"test/"
  path_train =path+"train/"

  phoneme = get_phoneme_dict()
  x_train,y_train = get_x_and_y(path_train,phoneme)         

  x_test,y_test = get_x_and_y(path_test,phoneme)

  max_len = get_max_feature_len(x_train)
  max_len=max(max_len,get_max_feature_len(x_test))

  # x_train = add_padding(x_train,max_len)
  # x_test = add_padding(x_test,max_len)

  return x_train, y_train, x_test, y_test



path="/content/drive/MyDrive/PRNN_A1/"
x_train, y_train, x_test, y_test = preprocessing_audio(path)
#n_test=16533
#n_train=57357

In [ ]:
# print(np.shape(x_train[13])[1])
# print(np.shape(x_test))
max_len1,max_len2 = get_max_feature_len(x_train)
t1,t2=get_max_feature_len(x_test)
max_len1=max(max_len1,t1)
max_len2=max(max_len2,t2)
print(max_len1)
print(max_len2)
max_len=10

20
44


In [ ]:
for i in range(1000):
  print(np.shape(x_train[i]))

In [ ]:
x_train.__len__()
np.shape(x_train[i])
print(i)

In [ ]:
xtrain = add_padding(x_train,max_len)
xtest = add_padding(x_test,max_len)
print(xtrain[0:2])
# print(xtest)

In [ ]:
print(np.shape(xtrain))
print(np.shape(xtest))
Xtrain = np.array(xtrain[0:1000])
Ytrain = np.array(y_train[0:1000])
Xtest = np.array(xtest[0:1000])
Ytest = np.array(y_test[0:1000])


(1000, 20, 10)
(1000, 20, 10)


In [ ]:
# def one_hot(y):
#   one_hot_y = np.zeros((y.size, y.max()+1)).astype("float32")
#   one_hot_y[np.arange(y.size), y] = 1
#   return one_hot_y
# Ytest=one_hot(Ytest)
# Ytrain=one_hot(Ytrain)

In [ ]:
Ytrain

LSTMs on the time-series using pyTorch

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable 

In [ ]:
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU avail")
else:
    device = torch.device("cpu")

In [ ]:
X_train = torch.from_numpy(Xtrain).float()
Y_train = torch.from_numpy(Ytrain).float()

In [ ]:
def create_sequences(data,label, seq_length):
    xs = []
    ys = []

    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length)]
        y = label[i+seq_length]
        x=np.array(x)
        xs.append(x)
        ys.append(y)

    return np.array(xs), np.array(ys)

In [ ]:
# xs,ys=create_sequences(Xtrain,Ytrain,3)
X_train = torch.from_numpy(Xtrain).float()
Y_train = torch.from_numpy(Ytrain).float()

# xs,ys=create_sequences(Xtest,Ytest,3)
X_test = torch.from_numpy(Xtest).float()
Y_test = torch.from_numpy(Ytest).float()

In [ ]:
print(X_train.size())
print(Y_train.size())

torch.Size([1000, 20, 10])
torch.Size([1000])


In [ ]:
class Predictor(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, drop_prob=0.2):
        super(Predictor, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.input_size = input_size
        
        
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.linear1 = nn.Linear(hidden_dim, 64)
        self.linear2 = nn.Linear(64, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        #x = x.long()
        #embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.linear1(out)
        out = self.linear2(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        #print(weight)
        hidden = ((weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()).to(torch.float32),
                      (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()).to(torch.float32))
        #print(hidden)
        return hidden

In [ ]:
train_data = TensorDataset(X_train, Y_train)
test_data = TensorDataset(X_test, Y_test)

batch_size = 64

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [ ]:
print(Y_test.shape)

torch.Size([1000])


In [ ]:
output_size = 2
#embedding_dim = 400
hidden_dim = 128
n_layers = 2
input_size = 10

model = Predictor(input_size, output_size, hidden_dim, n_layers)
#model.to(device)

lr=0.00002
#criterion = nn.BCELoss()
criterion = nn.MSELoss()
# criterion=nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [ ]:
epochs = 20
counter = 0
print_every = 1000
clip = 5
valid_loss_min = np.Inf
losses = []
model.train()
for i in range(epochs):
    h = model.init_hidden(batch_size)
    #h[0] = h[0].to(torch.float32)
    for inputs, labels in train_loader:
        counter += 1
        h = tuple([e.data for e in h])
        #inputs = inputs.to(torch.float64)
        print(h)
        print(len(h))
        inputs, labels = inputs, labels.to(device)

        model.zero_grad()
        output, h = model(inputs, h)

        print(output)
        loss = criterion(output, labels)
        loss.backward()
        # nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if counter%print_every == 0:
            val_h = model.init_hidden(batch_size)
            val_losses = []
            num_correct = 0
            num_yess = 0 
            model.eval()
            for inp, lab in test_loader:
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                #print(val_h[0].size)
                out, val_h = model(inp, val_h)
                # lab = lab.squeeze(-1)
                val_loss = criterion(out, lab.float())
                #val_loss = criterion(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
                pred = torch.round(output.squeeze())  # Rounds the output to 0/1
                
                correct_tensor = pred.eq(labels.float().view_as(pred))
                correct = np.squeeze(correct_tensor.cpu().numpy())
                if (pred == lab):
                  yess = 1
                  num_yess += 1
                else:
                  yess = 0
                num_correct += np.sum(correct)
                
            model.train()
            print("Epoch: {}/{}...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            test_acc = num_correct/len(test_loader.dataset)
            print("Test accuracy: {:.3f}%".format(test_acc*100))
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './state_dict.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)


(tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]), tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0

IndexError: ignored